# License Plate Recognition

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

## Training Set 1

In [ ]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset1 = '/content/Licplatesdetection_train.zip'

with ZipFile(dataset1,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

In [ ]:
import os
set1 = os.listdir('/content/license_plates_detection_train')
print(set1)

In [ ]:
len(set1)

### Reading Images in grayscale

In [ ]:
#Load and display the image
image_path = "/content/license_plates_detection_train/10.jpg"
image = cv2.imread(image_path)
image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image

In [ ]:
import os
import numpy as np
from PIL import Image

# Define the path to the image directory
image_dir = '/content/license_plates_detection_train'

data = []

# List all files in the directory
img_files = os.listdir(image_dir)

for img_file in img_files:
    # Construct the full path to the image file
    img_path = os.path.join(image_dir, img_file)

    # Open and process the image
    image = Image.open(img_path)
    image = image.resize((128, 128))
    image = image.convert('RGB')
    image = np.array(image)

    # Append the numpy array to the data list
    data.append(image)

# Convert the list to a numpy array
data = np.array(data)

print("Images successfully converted to numpy arrays")
print(f"Shape of data: {data.shape}")


In [ ]:
data[0]

In [ ]:
img = cv2.imread('/content/license_plates_detection_train/102.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

In [ ]:


# Define the path to the image directory
image_dir = '/content/license_plates_detection_train'

# List all files in the directory
img_files = os.listdir(image_dir)

# Sort the files to ensure consistent ordering
img_files.sort()

# Read and process the first 10 images
images = []
for img_file in img_files:
    # Construct the full path to the image file
    img_path = os.path.join(image_dir, img_file)

    # Read the image using OpenCV
    img = cv2.imread(img_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Append the grayscale image to the list
    images.append(gray)

# Display the first 10 images using Matplotlib
fig, axes = plt.subplots(2, 5, figsize=(20, 10))
axes = axes.ravel()

for i in range(10):
    axes[i].imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB), cmap='gray')
    axes[i].set_title(f'Image {i+1}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
images

In [ ]:
import imutils
import easyocr

### Applying the  filters and  finding the edges for localization


In [ ]:


# Define the path to the image directory
image_dir = '/content/license_plates_detection_train'

# List all files in the directory
img_files = os.listdir(image_dir)

# Sort the files to ensure consistent ordering
img_files.sort()

# Process and display the first 10 images
processed_images = []
for img_file in img_files:
    # Construct the full path to the image file
    img_path = os.path.join(image_dir, img_file)

    # Read the image using OpenCV
    img = cv2.imread(img_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply bilateral filtering for noise reduction
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)

    # Apply edge detection using Canny
    edged = cv2.Canny(bfilter, 30, 200)

    # Append the processed image to the list
    processed_images.append(edged)

# Display the first 10 processed images using Matplotlib
fig, axes = plt.subplots(2, 5, figsize=(20, 10))
axes = axes.ravel()

for i in range(10):
    axes[i].imshow(processed_images[i], cmap='gray')
    axes[i].set_title(f'Image {i+1}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
processed_images

In [ ]:
bfilter = cv2.bilateralFilter(processed_images[700], 11, 17, 17) #Noise reduction
edged = cv2.Canny(bfilter, 30, 200) #Edge detection
plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

### Finding Contours and applying the mask

In [ ]:

# Define the directory containing the images
image_dir = '/content/license_plates_detection_train'

# Get the list of all image file names in the directory
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# Initialize a list to store the locations of contours for each image
locations = []

for image_file in image_files[:900]:  # Limit to the first 900 images
    # Construct the full path to the image
    image_path = os.path.join(image_dir, image_file)

    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply edge detection
    edged = cv2.Canny(gray, 50, 200)

    # Find contours
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    # Store the location
    locations.append(location)

# Output the locations list
for i, location in enumerate(locations):
    print(f"Image {i+1}: {location}")


In [ ]:

# Defining the directory containing the images
image_dir = '/content/license_plates_detection_train'

# Get the list of all image file names in the directory
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# Process and store the results for each image
for i, image_file in enumerate(image_files):  # Limit to the first 900 images
    # Construct the full path to the image
    image_path = os.path.join(image_dir, image_file)

    # Read the image
    img = cv2.imread(image_path)

    # Converting the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection
    edged = cv2.Canny(gray, 50, 200)

    # Find contours
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        # Create a mask for the contour
        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        # Save or display the resulting image
        result_path = os.path.join(image_dir, f'result_{i+1}.png')
        cv2.imwrite(result_path, new_image)
        plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))
        plt.title(f"Processed Image {i+1}")
        plt.show()



## Training Set 2

In [ ]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset1 = '/content/Licplatesrecognition_train.zip'

with ZipFile(dataset1,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

In [ ]:
set2 = os.listdir('/content/license_plates_recognition_train')
print(set1)

In [ ]:
#Load and display the image
image_path = "/content/license_plates_recognition_train/10.jpg"
image = cv2.imread(image_path)
image

In [ ]:

# Defining the directory containing the images
image_dir = '/content/license_plates_recognition_train'

# Get the list of all image file names in the directory
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# Process and store the results for each image
for i, image_file in enumerate(image_files):  # Limit to the first 900 images
    # Construct the full path to the image
    image_path = os.path.join(image_dir, image_file)

    # Read the image
    img = cv2.imread(image_path)

    # Converting the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection
    edged = cv2.Canny(gray, 50, 200)

    # Find contours
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        # Create a mask for the contour
        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        # Save or display the resulting image
        result_path = os.path.join(image_dir, f'result_{i+1}.png')
        cv2.imwrite(result_path, new_image)
        plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))
        plt.title(f"Processed Image {i+1}")
        plt.show()



In [ ]:

# Define the directory containing the images
image_dir = '/content/license_plates_recognition_train'

# Get the list of all image file names in the directory
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# Process and store the results for each image
images2=[]
for i, image_file in enumerate(image_files[:900]):  # Limit to the first 900 images
    # Construct the full path to the image
    image_path = os.path.join(image_dir, image_file)

    # Read the image
    img = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply edge detection
    edged = cv2.Canny(gray, 50, 200)

    # Find contours
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break

    if location is not None:
        # Create a mask for the contour
        mask = np.zeros(gray.shape, np.uint8)
        cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        # Find the bounding box of the masked area
        (x, y) = np.where(mask == 255)
        if len(x) > 0 and len(y) > 0:  # Ensure there are white pixels in the mask
            (x1, y1) = (np.min(x), np.min(y))
            (x2, y2) = (np.max(x), np.max(y))
            cropped_image = gray[x1:x2+1, y1:y2+1]
            images2.append(cropped_image)

            # Save the resulting cropped image
            result_path = os.path.join(image_dir, f'cropped_result_{i+1}.png')
            cv2.imwrite(result_path, cropped_image)
        else:
            print(f"No contour found for image {image_file}")
    else:
        print(f"No contour with 4 corners found for image {image_file}")


In [ ]:
plt.imshow(cv2.cvtColor(images2[100], cv2.COLOR_BGR2RGB))


In [ ]:
reader = easyocr.Reader(['en'])
result = reader.readtext(images2[100])
result

In [ ]:
if result:
  text = ' '.join([detection[1] for detection in result])
  print(f"Text detected in image {i+1}: {text}")
else:
  print(f"No text detected in image {i+1}")

In [ ]:

reader = easyocr.Reader(['en'])

# Iterate through the images and detect text
for i, image in enumerate(images2):
  result = reader.readtext(image)
  if result:
    text = ' '.join([detection[1] for detection in result])
    print(f"Text detected in image {i+1}: {text}")
  else:
    print(f"No text detected in image {i+1}")
